In [15]:
# Import packages
import pandas as pd

# Read CSV from path
path = "data/mapdataall.csv"

fire_data = pd.read_csv(path)

In [16]:
# Select and rename columns from original csv data
fire_data_columns = ["incident_name", "incident_id", "incident_county", 
                     "incident_longitude", "incident_latitude", "incident_acres_burned", 
                     "incident_dateonly_created", "incident_dateonly_extinguished"]

fire_columns_rename = fire_data[fire_data_columns].rename(columns={
    "incident_name":"Name of Fire",
    "incident_id":"Fire ID",
    "incident_county": "County", 
    "incident_longitude": "Longitude", 
    "incident_latitude": "Latitude", 
    "incident_acres_burned": "Acres Burned", 
    "incident_dateonly_created": "Date Created", 
    "incident_dateonly_extinguished": "Date Extinguished"})

In [17]:
#Clean and only look at Central California data
fire_data_clean = fire_columns_rename.dropna()

central_county = ["Madera", "Kern", "San Benito", "Monterey", "San Joaquin", "San Luis Obispo", "Santa Barbara", "Tuolumne", "Merced", "Fresno", "Mariposa", "Stanislaus", "Kings", "Colusa"]
central_county_data = fire_data_clean[fire_data_clean["County"].isin(central_county)]
central_county_data

,Name of Fire,Fire ID,County,Longitude,Latitude,Acres Burned,Date Created,Date Extinguished
4,Gold Fire,357ffc13-bef9-48eb-810f-c5de851972eb,Madera,-119.635004,37.116295,274.0,2013-04-30,2013-05-01
9,Tres Pinos Fire,ca68f3c4-4906-4952-b794-4531d7da8369,San Benito,-120.937494,37.160346,354.0,2013-05-03,2013-05-03
16,Lilly 2 Fire,2b765a08-c770-4b19-baa8-00dffe3fa838,Madera,-119.791700,37.192700,91.0,2013-05-19,2013-05-20
17,Cottontail Fire,f17c523c-bf01-4245-899b-7c2331096dae,San Luis Obispo,-120.913750,35.498640,35.0,2013-05-20,2013-05-20
20,Shelton Fire,0b51d59b-457d-4a62-b066-55c2b8d400e2,San Joaquin,-120.903250,38.049930,303.0,2013-05-30,2013-05-30
...,...,...,...,...,...,...,...,...
1998,Anzar Fire,367d210d-571c-4066-a078-44f3e162d8f4,San Benito,-121.604728,36.892768,104.0,2022-07-21,2022-07-27
2011,Springs Fire,bb08947c-f887-4538-bb5e-1d5750b54249,Tuolumne,-120.508060,37.843490,51.0,2022-08-03,2022-08-04
2018,Rail Fire,b2407838-9dd2-45b8-a956-899bf6891b98,Kern,-118.629224,35.288264,250.0,2022-08-14,2022-08-17
2045,Rosa Fire,90ff8cb6-8361-4442-8646-b8846bf2f7ea,San Luis Obispo,-121.012313,35.580125,16.0,2022-09-06,2022-09-06
